In [4]:
import os
import numpy as np
import tensorflow as tf
from data_loader import DataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import lstm_resnet

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/train/images/images",
        mask_dir="../Unet/data_v3_processed/train/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True,
    )
        
    val_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/val/images/images",
        mask_dir="../Unet/data_v3_processed/val/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True
    )

    return train_gen, val_gen

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=4, steps_per_epoch=1000, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = lstm_resnet(256, 256, 3, 15)
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)

        model_path = os.path.join("models", f"lstm_resnet_run{run_number+1}.h5")
        os.makedirs("models", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [42, 123, 456, 789, 999]

for run in range(5):
    # CREATE FRESH GENERATORS FOR EACH RUN - this fixes the state issue
    train_generator, val_generator = create_data_generators()
    
    model_path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    
    tf.keras.backend.clear_session()


Training Run 1/5 (Seed: 42)
Epoch 1/4
1000/1000 [==============================] - 419s 409ms/step - loss: 0.0212 - f1: 0.6869 - val_loss: 0.0282 - val_f1: 1.8927e-04
Epoch 2/4
1000/1000 [==============================] - 409s 409ms/step - loss: 0.0020 - f1: 0.9254 - val_loss: 0.0160 - val_f1: 0.5789
Epoch 3/4
1000/1000 [==============================] - 409s 408ms/step - loss: 0.0011 - f1: 0.9515 - val_loss: 0.0088 - val_f1: 0.7337
Epoch 4/4
1000/1000 [==============================] - 409s 408ms/step - loss: 0.0010 - f1: 0.9514 - val_loss: 0.0184 - val_f1: 0.7302

Training Run 2/5 (Seed: 123)
Epoch 1/4
1000/1000 [==============================] - 419s 409ms/step - loss: 0.0277 - f1: 0.6168 - val_loss: 0.0350 - val_f1: 0.0333
Epoch 2/4
1000/1000 [==============================] - 409s 408ms/step - loss: 0.0020 - f1: 0.9262 - val_loss: 0.0104 - val_f1: 0.7286
Epoch 3/4
1000/1000 [==============================] - 408s 407ms/step - loss: 0.0012 - f1: 0.9456 - val_loss: 0.0113 - val_f1: